In [73]:
import numpy as np
from pybullet_environment import *

In [72]:
env = RobotEnvironment(manipulator = False)

argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.86.05
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.86.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [4]:
def euler_to_rotation_matrix(yaw, pitch, roll):
        
    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                [np.sin(yaw), np.cos(yaw), 0],
                [0, 0, 1]])

    Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                [0, 1, 0],
                [-np.sin(pitch), 0, np.cos(pitch)]])

    Rx = np.array([[1, 0, 0],
                [0, np.cos(roll), -np.sin(roll)],
                [0, np.sin(roll), np.cos(roll)]])

    R = Rz @ (Ry @ Rx)
    
    return R

In [5]:
def define_vertices(pos, ori, dims):

    l, b, h = dims[0], dims[1], dims[2]

    local_vertices = np.array([[-l/2, -b/2, -h/2, 1],
                                [ l/2, -b/2, -h/2, 1],
                                [ l/2,  b/2, -h/2, 1],
                                [-l/2,  b/2, -h/2, 1],
                                [-l/2, -b/2,  h/2, 1],
                                [ l/2, -b/2,  h/2, 1],
                                [ l/2,  b/2,  h/2, 1],
                                [-l/2,  b/2,  h/2, 1]])
    
    rot = euler_to_rotation_matrix(ori[0], ori[1], ori[2])

    T = np.eye(4)
    T[:3, :3] = rot
    T[:3, 3] = pos[:]

    global_vertices = (T @ (local_vertices.T)).T

    return global_vertices[:, :-1]  

In [33]:
def find_furthest_vertex(direction, vertices):

    local_vertices = vertices - (np.mean(vertices, axis = 0)[np.newaxis, :])

    dot_product = (local_vertices @ direction)[:, 0]
    
    furthest_vertex = vertices[np.argmax(dot_product)]

    return furthest_vertex

def support_mapping(direction, X, Y):

    return find_furthest_vertex(direction, X) - find_furthest_vertex(-direction, Y)
        

In [40]:
def unit_vector(v):

    norm = np.linalg.norm(v)
    return v / norm

In [69]:
def update_simplex(simplex, X, Y):

    # Return 0 means that he origin lies on either a vertex, an edge, or a face

    if len(simplex) == 0:
        direction = np.array([[1., 0., 0.]]).T
        next_point = support_mapping(direction, X, Y)
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)
    
    elif len(simplex) == 1:
        A = simplex[0]
        direction = -A
        if np.linalg.norm(direction) == 0:
            return 0
        next_point = support_mapping(unit_vector(direction), X, Y)
        if np.dot(next_point, direction) < 0:
            return False
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)

    elif len(simplex) == 2:
        A, B = simplex[0], simplex[1]
        BA = A - B
        direction = np.cross(np.cross(BA, B), BA)
        if np.linalg.norm(direction) == 0:
            return 0
        next_point = support_mapping(unit_vector(direction), X, Y)
        if np.dot(next_point, direction) < 0:
            return False
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)

    elif len(simplex) == 3:
        A, B, C = simplex[0], simplex[1], simplex[2]
        BA = A - B
        CA = A - C
        direction = np.cross(CA, BA)
        if np.dot(direction, C) > 0:
            direction = -direction
        if np.linalg.norm(direction) == 0:
            return 0
        next_point = support_mapping(unit_vector(direction), X, Y)
        if np.dot(next_point, direction) < 0:
            return False
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)

    elif len(simplex) == 4:
        
        A, B, C, D = simplex[0], simplex[1], simplex[2], simplex[3]
        
        DA = D - A
        DB = D - B
        DC = D - C
        centroid = (A + B + C + D)/4
        to_centroid = centroid - D
        faces = [(DA, DB), (DB, DC), (DC, DA)]

        direction = None

        for face in faces:
            normal = np.cross(face[0], face[1])
            if np.dot(normal, to_centroid) > 0:
                normal = -normal
            if np.dot(normal, D) < 0:
                direction = normal.copy()

        if type(direction) == type(None):
            return True
        
        simplex.pop(0)
        next_point = support_mapping(unit_vector(direction), X, Y)
        if np.dot(next_point, direction) < 0:
            return False
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)

In [80]:
env.client_id.removeBody(A_id)
env.client_id.removeBody(B_id)

X_pos = np.array([0.2, 0.1, 0.5])
X_ori = np.array([0., 0., 0.])
X_size = np.array([0.5, 0.3, 0.2])

Y_pos = np.array([0.56, 0.11, 0.51])
Y_ori = np.array([0.1, 0.3, -0.4])
Y_size = np.array([0.15, 0.3, 0.08])

vuid = env.client_id.createVisualShape(p.GEOM_BOX, 
                                       halfExtents = X_size/2,
                                       rgbaColor = np.hstack([env.colors['yellow'], np.array([1.0])]))

A_id = env.client_id.createMultiBody(baseVisualShapeIndex = vuid, 
                                       basePosition = X_pos, 
                                       baseOrientation = env.client_id.getQuaternionFromEuler(X_ori))

vuid = env.client_id.createVisualShape(p.GEOM_BOX, 
                                       halfExtents = Y_size/2,
                                       rgbaColor = np.hstack([env.colors['red'], np.array([1.0])]))

B_id = env.client_id.createMultiBody(baseVisualShapeIndex = vuid, 
                                       basePosition = Y_pos, 
                                       baseOrientation = env.client_id.getQuaternionFromEuler(Y_ori))

# Form the vertices of the shapes:
X = define_vertices(X_pos, X_ori, X_size)
Y = define_vertices(Y_pos, Y_ori, Y_size)
simplex = []

out = update_simplex(simplex, X, Y)
print(out)

True


In [75]:
env.client_id.removeBody(A_id)
env.client_id.removeBody(B_id)

In [44]:
# Form the vertices of the shapes:
X = define_vertices(X_pos, X_ori, X_size)
Y = define_vertices(Y_pos, Y_ori, Y_size)

# Pick a random vertex on the Minkowski Difference:
direction = np.array([[1., 0., 0.]]).T
A = support_mapping(direction, X, Y)
# Remember the support mapping gives us the furthest vertex in a direction from the centroid of the minkowski difference,
# in terms of the Original Origin (that is, global coordinates, and NOT local to the Minkowski Difference Shape)

# Pick second vertex towards origin:
direction = -A
if np.linalg.norm(direction) == 0:
    print(True)
B = support_mapping(unit_vector(direction), X, Y)
if np.dot(B, direction) < 0:
    print(False)

# Pick third vertex towards origin:
BA = A - B
direction = np.cross(np.cross(BA, B), BA)
if np.linalg.norm(direction) == 0:
    print(True)
C = support_mapping(unit_vector(direction), X, Y)
if np.dot(C, direction) < 0:
    print(False)

# Pick fourth vertex towards origin:
CA = A - C
direction = np.cross(CA, BA)
if np.dot(direction, C) > 0:
    direction = -direction
if np.linalg.norm(direction) == 0:
    print(True)
D = support_mapping(unit_vector(direction), X, Y)
if np.dot(D, direction) < 0:
    print(False)

# In the above 3 snippets, if the origin lies on either a vertex, an edge, or a face, the direction will be zero.

# Pick next vertex from tetrahedron:

contains_origin = False
direction = None

DA = D - A
DB = D - B
DC = D - C
centroid = (A + B + C + D)/4
to_centroid = centroid - D
# Face 1:
normal = np.cross(DA, DB)
if np.dot(normal, to_centroid) > 0:
    normal = -normal
if np.dot(normal, D) < 0:
    direction = normal.copy()
# Face 2:
normal = np.cross(DB, DC)
if np.dot(normal, to_centroid) > 0:
    normal = -normal
if np.dot(normal, D) < 0:
    direction = normal.copy()
# Face 3:
normal = np.cross(DC, DA)
if np.dot(normal, to_centroid) > 0:
    normal = -normal
if np.dot(normal, D) < 0:
    direction = normal.copy()

if type(direction) == type(None):
    contains_origin = True


<>:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_6924/821103382.py:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  (DA, DB) (DB, DC) (DC, DA)
/tmp/ipykernel_6924/821103382.py:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  (DA, DB) (DB, DC) (DC, DA)
/tmp/ipykernel_6924/821103382.py:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  (DA, DB) (DB, DC) (DC, DA)
/tmp/ipykernel_6924/821103382.py:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  (DA, DB) (DB, DC) (DC, DA)


NameError: name 'DA' is not defined

In [43]:
np.cross(np.cross([1, 2, 3], [1, 2, 3]), [1, 2, 3])

array([0, 0, 0])

In [32]:


print(support_mapping([1., -0.1, -0.5], X, Y))

[ 0.33298288 -0.30517837 -0.11155711]


In [46]:
l = [1, 2, 3]
l.pop(0)
print(l)

[2, 3]
